In [772]:
'''
All essential imports
'''

import numpy as np
from sklearn.model_selection import train_test_split
import copy
import pandas as pd

In [773]:
'''
Data to train on, chat gpt generated columns etc.
'''


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

df = pd.read_csv(url, names=column_names, sep=', ', na_values='?', engine='python')

df.dropna(inplace=True)

X_raw = df.drop('income', axis=1)

y = (df['income'] == '>50K').astype(int).values

X_processed = pd.get_dummies(X_raw, drop_first=True)
X = X_processed.values
epsilon = 1e-7
X = X.astype(np.float64)

In [774]:
X = (X - X.mean(axis=0)) / (X.std(axis=0) + epsilon)
X.shape, y.shape

((30162, 96), (30162,))

In [775]:
'''
Transform X and y so that their shape will be suitable  for Deep Learning Algorithm

X.shape <- (n[0], m)
y.shape <- (1, m)
m - number of training examples,
n - number of features in particular layer
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  ## Train test split there could be nothing wrong
x_train_shape = (X_train.shape[1], X_train.shape[0])
x_test_shape = (X_test.shape[1], X_test.shape[0])
m = X_train.shape[0]
n = X_test.shape[0]

X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape(1, -1)
y_test = y_test.reshape(1, -1)
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)

assert y_train.shape == (1, m), "y_train has wrong shape"
assert X_train.shape == x_train_shape, "X_train has wrong shape"
assert y_test.shape == (1, n), "y_test has wrong shape"
assert X_test.shape == x_test_shape, "X_test has wrong shape"
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((96, 24129), (96, 6033), (1, 24129), (1, 6033))

In [776]:
assert (X_train.shape, X_test.shape, y_train.shape, y_test.shape) == ((96, 24129), (96, 6033), (1, 24129), (1, 6033)), "Shapes are NOT correct"


In [777]:
'''
Declare function, that will determine network's number of layers and their sizes
'''
def create_layers(k):
  assert isinstance(k, list) and all(isinstance(i, int) for i in k) and len(k)>1, "input must be list of integers of size bigger than 1"
  return k

In [778]:
'''
Parameters initialization:
input:
list of integers()
output:
W <- weights
b <- bias
'''

def initialize_parameters(layers_list):
  parameters = {}
  layers = len(layers_list)
  ##  this loop | is correct
  ##            V
  for l in range(1, layers):
    parameters['W' + str(l)] = np.random.randn(layers_list[l], layers_list[l-1]) * np.sqrt(2. / layers_list[l-1])
    parameters['b' + str(l)] = np.zeros((layers_list[l], 1))
  assert len(parameters) == (layers-1) * 2
  return parameters


In [779]:
'''
activation functions
'''
def sigmoid(x):
  s = 1 / (1 + np.exp(-x))
  return s
def relu(x):
  s = np.maximum(0, x)
  return s
def leaky_relu(x):
  s = np.maximum(0.01*x, x)
  return s
def tanh(x):
  s = np.tanh(x)
  return s


In [780]:
'''
Forward Pass
'''
def forward(A , W, b):
  Z = np.dot(W, A) + b
  cache = (A, W, b)
  return Z, cache


In [781]:
def activation_forward(A_prev, W, b, activation):
  assert activation in ('relu', 'l_relu', 'sigmoid', 'tanh')
  Z, lin_cache = forward(A_prev, W, b)
  match activation:
    case 'relu':
      A = relu(Z)
    case 'sigmoid':
      A = sigmoid(Z)
    case 'tanh':
      A = tanh(Z)
    case 'l_relu':
      A = leaky_relu(Z)
  cache = (lin_cache, (Z))
  return A, cache


In [782]:
def forward_loop(X, parameters, activation_list):
  caches = []
  A = X
  L = len(parameters)//2
  ## crucial for editing
  act_length = len(activation_list)
  assert  act_length == L, "Not enough activation functions"
  ##
  for l in range(1, L+1):
    A_prev = A
    A, cache = activation_forward(A_prev, parameters['W'+str(l)], parameters['b' + str(l)], activation_list[l-1])
    caches.append(cache)
  return A, caches

In [783]:
def compute_cost(A, y):
  m = y.shape[1]
  cost = - (np.dot(y, np.log(A).T) + np.dot((1 - y), np.log(1 - A).T)) / m
  cost = np.squeeze(cost)
  assert cost >= 0, "Smth wrong??? Check your cost function dawg"
  return cost


In [784]:
#TODO Backprop, backprop loop, prediction, check if data is standardized, think about cost functions, and better implementing cache etc.

In [785]:

def reluDerivative(x, activation):
  match activation:
    case 'relu':
      x = (x > 0) * 1
      return x
    case 'l_relu':
      dx = np.ones_like(x)
      dx[x <= 0] = 0.01
      return dx

def compute_derivative(Z, activation):
  match activation:
    case 'sigmoid':
      A = sigmoid(Z)
      return A*(1-A)
    case 'relu':
      return reluDerivative(Z, 'relu')
    case 'l_relu':
      return reluDerivative(Z, 'l_relu')
    case 'tanh':
      A = tanh(Z)
      return 1 - A * A

In [786]:
def backward(dZ, cache, m):
  A_prev, W, b = cache
  dW = np.dot(dZ, A_prev.T) / m
  db = np.sum(dZ, axis=1, keepdims=True)/m
  dA_prev = np.dot(W.T, dZ)
  return dA_prev, dW, db
def activation_backward(dA, cache, activation, m):
  linear_cache, activation_cache = cache # A_prev, W, b, Z
  dZ = dA * compute_derivative(activation_cache, activation)
  dA_prev, dW, db = backward(dZ, linear_cache, m)
  return dA_prev, dW, db


In [787]:
def backward_loop(A, y, caches, activation_list, m):
  grads = {}
  L = len(caches)
  y = y.reshape(A.shape)
  dA = - (np.divide(y, A) - np.divide(1 - y, 1 - A))
  current_cache = caches[L-1]
  linear_cache, activation_cache = current_cache # A[L-1], W[L] , b
  dZ = A - y # skrot
  A_prev, W, b = linear_cache
  dW = np.dot(dZ, A_prev.T) / m
  db = np.sum(dZ, axis=1, keepdims=True) / m
  dA_prev = np.dot(W.T, dZ)
  grads["dA" + str(L-1)] = dA_prev
  grads["dW" + str(L)] = dW
  grads["db" + str(L)] = db
  for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev, dW, db = activation_backward(dA_prev, current_cache, activation_list[l], m)
    grads["dA"+ str(l)] = dA_prev
    grads["dW" + str(l + 1)] = dW
    grads["db" + str(l + 1)] = db
  return grads

In [788]:
## Ready
def update_parameters(params, grads, lr=0.1):
  parameters = copy.deepcopy(params)
  L = len(parameters)//2
  for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - lr * grads["dW"+ str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - lr * grads["db"+ str(l+1)]
  return parameters

In [789]:
def nn_model(X, y, layers, n_iterations, lr, print_cost = False):
  m = X.shape[1]
  print(m)
  layers = create_layers(layers)
  K = len(layers) -1
  parameters = initialize_parameters(layers)
  activation_list = ['relu'] * (K-1)
  activation_list.append('sigmoid')
  print(activation_list)
  for iteration in range(n_iterations):
    A, caches = forward_loop(X, parameters, activation_list)
    cost = compute_cost(A, y)
    grads = backward_loop(A, y, caches, activation_list, m)
    parameters = update_parameters(parameters, grads, lr)
    if iteration % 50 == 0 and print_cost:
      print(f'iteration: {iteration}/{n_iterations}, cost: {cost}')
  return parameters

In [790]:
model = nn_model(X_train, y_train, [96, 30, 40, 20, 30, 40, 20, 1], 1000, 0.01, print_cost=True)

24129
['relu', 'relu', 'relu', 'relu', 'relu', 'relu', 'sigmoid']
iteration: 0/1000, cost: 1.496665264263974
iteration: 50/1000, cost: 0.6036106324297397
iteration: 100/1000, cost: 0.5438142415006404
iteration: 150/1000, cost: 0.5137508624236845
iteration: 200/1000, cost: 0.49034160131963755
iteration: 250/1000, cost: 0.4707848460933471
iteration: 300/1000, cost: 0.4545928690020772
iteration: 350/1000, cost: 0.4409048058262162
iteration: 400/1000, cost: 0.42940614987072007
iteration: 450/1000, cost: 0.41973160396748066
iteration: 500/1000, cost: 0.4115750652263683
iteration: 550/1000, cost: 0.40458751881882504
iteration: 600/1000, cost: 0.39856931930143147
iteration: 650/1000, cost: 0.39329167727608955
iteration: 700/1000, cost: 0.38866473757015385
iteration: 750/1000, cost: 0.38451646759101343
iteration: 800/1000, cost: 0.3807812248032603
iteration: 850/1000, cost: 0.3773758113615056
iteration: 900/1000, cost: 0.3743130016125845
iteration: 950/1000, cost: 0.37149676266835835


In [792]:
def predict(X, parameters):
  L = len(parameters) // 2
  activation_list = ['relu'] * (L-1)
  activation_list.append('sigmoid')
  A, _ = forward_loop(X, parameters, activation_list)
  predictions = (A > 0.5).astype(int)
  return predictions

In [793]:
predictions_train = predict(X_train, model)
predictions_test = predict(X_test, model)
train_accuracy = np.mean(predictions_train == y_train) * 100
test_accuracy = np.mean(predictions_test == y_test) * 100
print(f"Accuracy on Train set: {train_accuracy:.2f}%")
print(f"Accuracy on Test set: {test_accuracy:.2f}%")

Accuracy on Train set: 82.53%
Accuracy on Test set: 82.36%
